<a href="https://colab.research.google.com/github/MicaelaRomeroC3/sotto/blob/main/Sottotempo_Registro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install workalendar
!pip install pytz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.8 MB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732000 sha256=fc0b52b214f1bfb7fa46e90d34012f8224b7ef07cebd2988c572e569a356dca6
  Stored in directory: /root/.cache/pip/wheels/8f/bd/f9/5c4c39b529e0322b08979e1c465e203218bc2cca75d20f7df5
Successfully built pymeeus


In [196]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Extraer Canales Realizados y crear Hoja REGISTRO REALIZADA**

# Teléfono Realizada

In [197]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dftelefono = pd.read_excel(file_path, sheet_name='Teléfono')


dftelefono.head()

,EMPRESA,REALIZADA,FECHA
0,ACER,REALIZADO,2025-05-06
1,ACER,REALIZADO,2025-05-06
2,ACER,REALIZADO,2025-05-06
3,ACER,REALIZADO,2025-05-06
4,ACER,REALIZADO,2025-05-07


**Contar previamente la cantidad de realizado presentes**

In [198]:
# Eliminar espacios antes y después en los nombres de las columnas
dftelefono.columns = dftelefono.columns.str.strip()

# Eliminar espacios antes y después en todo el contenido tipo string
dftelefono = dftelefono.applymap(lambda x: x.strip() if isinstance(x, str) else x)

conteo = dftelefono['REALIZADA'].str.strip().str.lower()
total_REALIZADO = (conteo == 'realizado').sum()
print(f"Total de 'REALIZADO': {total_REALIZADO}")


Total de 'REALIZADO': 1607


<ipython-input-198-d52feb9768b7>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dftelefono = dftelefono.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [199]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dftelefono['EMPRESA'] = dftelefono['EMPRESA'].str.strip()

dftelefono['EMPRESA'] = (dftelefono['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Corregir las empresas que están mal escritas**

In [200]:
# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas'
}

# Aplica los reemplazos
dftelefono['EMPRESA'] = dftelefono['EMPRESA'].replace(reemplazos)

dftelefono['EMPRESA']
print(dftelefono[dftelefono['EMPRESA'] == 'canal de isabel ii'])


                EMPRESA  REALIZADA      FECHA
320  canal de isabel ii  REALIZADO 2025-05-06
321  canal de isabel ii  REALIZADO 2025-05-06
322  canal de isabel ii  REALIZADO 2025-05-06
323  canal de isabel ii  REALIZADO 2025-05-07
324  canal de isabel ii  REALIZADO 2025-05-07
325  canal de isabel ii  REALIZADO 2025-05-07
326  canal de isabel ii  REALIZADO 2025-05-07
327  canal de isabel ii  REALIZADO 2025-05-08
328  canal de isabel ii  REALIZADO 2025-05-08
329  canal de isabel ii  REALIZADO 2025-05-08
330  canal de isabel ii  REALIZADO 2025-05-08
331  canal de isabel ii  REALIZADO 2025-05-09
332  canal de isabel ii  REALIZADO 2025-05-09
333  canal de isabel ii  REALIZADO 2025-05-09
334  canal de isabel ii  REALIZADO 2025-05-12
335  canal de isabel ii  REALIZADO 2025-05-12
336  canal de isabel ii  REALIZADO 2025-05-12
337  canal de isabel ii  REALIZADO 2025-05-12
338  canal de isabel ii  REALIZADO 2025-05-13
339  canal de isabel ii  REALIZADO 2025-05-13
340  canal de isabel ii  REALIZADO

**Generar Hoja Registro para Realizada Teléfono**

In [201]:
# Limpiar espacios adelante y atrás en todas las columnas de tipo texto
dftelefono = dftelefono.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Estandarizar columna 'REALIZADA'
dftelefono['realizada_limpia'] = dftelefono['REALIZADA'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dftelefono[dftelefono['realizada_limpia'].isin(['realizado'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Telefono'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dftelefono = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dftelefono = dftelefono[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dftelefono.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dftelefono


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Telefono,Realizada,48
1,adeslas,Telefono,Realizada,11
2,aeda homes,Telefono,Realizada,22
3,aegon salud,Telefono,Realizada,13
4,aegon vida,Telefono,Realizada,3
...,...,...,...,...
73,ups,Telefono,Realizada,15
74,viatris,Telefono,Realizada,37
75,whirlpool,Telefono,Realizada,13
76,xiaomi,Telefono,Realizada,12


In [202]:
total_cantidad = dftelefono['cantidad'].sum()
print("Total de la columna 'cantidad':", total_cantidad)


Total de la columna 'cantidad': 1607


# Mail Realizada

In [203]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfmail = pd.read_excel(file_path, sheet_name='Mail')


dfmail.head()

,EMPRESA,ESTADO CORREO,FECHA
0,ACER,REALIZADO,2025-05-07
1,ACER,REALIZADO,2025-05-07
2,AEDA HOMES,REALIZADO,2025-05-07
3,AGBAR,REALIZADO,2025-05-07
4,AGBAR,REALIZADO,2025-05-07


**Contar la cantidad de enviado**

In [204]:
# Eliminar espacios antes y después en todo el contenido tipo string
dfmail = dfmail.applymap(lambda x: x.strip() if isinstance(x, str) else x)

enviados = (dfmail['ESTADO CORREO'].str.strip().str.lower() == 'realizado').sum()
print(f"Cantidad de 'realizado': {enviados}")

Cantidad de 'realizado': 483


<ipython-input-204-fb3e6651b9ff>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfmail = dfmail.applymap(lambda x: x.strip() if isinstance(x, str) else x)


**Corregir las empresas que están mal escritas**

In [205]:
#Pasar a minúscula
dfmail['EMPRESA'] = dfmail['EMPRESA'].str.lower()

# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas'
}

# Aplica los reemplazos
dfmail['EMPRESA'] = dfmail['EMPRESA'].replace(reemplazos)

dfmail['EMPRESA']

,EMPRESA
0,acer
1,acer
2,aeda homes
3,agbar
4,agbar
...,...
478,viatris
479,viatris
480,viatris
481,whirlpool


**Pasar todos los datos de Empresa a minúscula**

In [206]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfmail['EMPRESA'] = dfmail['EMPRESA'].str.strip()

dfmail['EMPRESA'] = (dfmail['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Generar hoja Registro de Realizadas Mail**

In [207]:
# Estandarizar columna 'REALIZADA'
dfmail['realizada_limpia'] = dfmail['ESTADO CORREO'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dfmail[dfmail['realizada_limpia'].isin(['realizado'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Mail/formulario'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dfmail = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dfmail = dfmail[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dfmail.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dfmail.head()

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Mail/formulario,Realizada,12
1,adeslas,Mail/formulario,Realizada,4
2,aeda homes,Mail/formulario,Realizada,4
3,agbar,Mail/formulario,Realizada,12
4,asisa,Mail/formulario,Realizada,4


# WEB Realizada

In [208]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfweb = pd.read_excel(file_path, sheet_name='Web')


dfweb.head()

,EMPRESA,ESTADO NAVEGACIÓN,FECHA
0,ACER,ENCONTRADA,2025-05-13
1,ACER,ENCONTRADA,2025-05-14
2,ACER,ENCONTRADA,2025-05-15
3,ACER,ENCONTRADA,2025-05-16
4,ADESLAS,ENCONTRADA,2025-05-13


In [209]:
# Eliminar espacios antes y después en todo el contenido tipo string
dfweb = dfweb.applymap(lambda x: x.strip() if isinstance(x, str) else x)

enviados = (dfweb['ESTADO NAVEGACIÓN'].str.strip().str.lower() == 'encontrada').sum()
print(f"Cantidad de 'encontrada': {enviados}")

Cantidad de 'encontrada': 36


<ipython-input-209-1c4807fbab1d>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfweb = dfweb.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [210]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfweb['EMPRESA'] = dfweb['EMPRESA'].str.strip()

dfweb['EMPRESA'] = (dfweb['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Corregir las empresas que están mal escritas**

In [211]:
# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas'
}

# Aplica los reemplazos
dfweb['EMPRESA'] = dfweb['EMPRESA'].replace(reemplazos)

dfweb['EMPRESA']

,EMPRESA
0,acer
1,acer
2,acer
3,acer
4,adeslas
5,adeslas
6,aeda homes
7,aeda homes
8,aeda homes
9,asisa


In [212]:
# Estandarizar columna 'REALIZADA'
dfweb['realizada_limpia'] = dfweb['ESTADO NAVEGACIÓN'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dfweb[dfweb['realizada_limpia'].isin(['encontrada'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Web'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dfweb = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dfweb = dfweb[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dfweb.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dfweb

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Web,Realizada,4
1,adeslas,Web,Realizada,2
2,aeda homes,Web,Realizada,3
3,asisa,Web,Realizada,2
4,asus,Web,Realizada,2
5,aurgi,Web,Realizada,2
6,avis,Web,Realizada,2
7,beko,Web,Realizada,1
8,bosch,Web,Realizada,1
9,bosch talleres,Web,Realizada,4


# Facebook y Twitter Realizada

In [213]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfredes = pd.read_excel(file_path, sheet_name='Redes')


dfredes.head()

,EMPRESA,tipo_contacto,ESTADO RR SS,FECHA
0,ASUS,FACEBOOK,REALIZADO,2025-05-13
1,AURGI,FACEBOOK,REALIZADO,2025-05-13
2,AVIS,FACEBOOK,REALIZADO,2025-05-13
3,BEKO,FACEBOOK,REALIZADO,2025-05-13
4,BOSCH,FACEBOOK,REALIZADO,2025-05-13


**Corregir las empresas que están mal escritas**

In [214]:
#Pasar a minúscula
dfredes['EMPRESA'] = dfredes['EMPRESA'].str.lower()

# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas'
}

# Aplica los reemplazos
dfredes['EMPRESA'] = dfredes['EMPRESA'].replace(reemplazos)

dfredes['EMPRESA']

,EMPRESA
0,asus
1,aurgi
2,avis
3,beko
4,bosch
5,bosch talleres
6,bridgestone
7,canal de isabel ii
8,dia fidelización
9,continental


**Recuento de la cantidad por canal**

In [215]:
# Eliminar espacios antes y después en todo el contenido tipo string
dfredes = dfredes.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Estandarizamos texto
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].astype(str).str.strip().str.lower()
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].fillna('').str.strip().str.lower()
dfredes['tipo_contacto'] = dfredes['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo los "enviado"
df_enviados = dfredes[dfredes['estado_rr_ss_limpio'] == 'realizado']

# Verificamos si el DataFrame tiene datos antes de proceder
if not df_enviados.empty:
    conteo = df_enviados['tipo_contacto'].value_counts().loc[['Facebook', 'Twitter']].to_dict()
    print(f"Cantidad de 'enviado': {conteo}")
else:
    print("No hay datos disponibles en el DataFrame para procesar.")

# Mostramos resultado
print(f"Cantidad de 'enviado': {conteo}")


Cantidad de 'enviado': {'Facebook': 40, 'Twitter': 6}
Cantidad de 'enviado': {'Facebook': 40, 'Twitter': 6}


<ipython-input-215-8e5f2dac577c>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfredes = dfredes.applymap(lambda x: x.strip() if isinstance(x, str) else x)


**Ejecutar para Facebook**

In [216]:
# Estandarizamos texto
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].str.strip().str.lower()
dfredes['tipo_contacto'] = dfredes['tipo_contacto'].str.strip().str.capitalize()  # o 'tipo_contacto' si es otro nombre

# Filtramos solo "enviado" por canal
df_facebook = dfredes[(dfredes['estado_rr_ss_limpio'] == 'realizado') & (dfredes['tipo_contacto'] == 'Facebook')]

# Agregar columnas fijas
for df_temp in [df_facebook]:
    df_temp['tipo_de_acción'] = 'Realizada'
    df_temp['tipo_contacto'] = df_temp['tipo_contacto']

# Agrupar y contar
df_facebook = df_facebook.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_facebook.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar ambos resultados
print("📘 Facebook:")
df_facebook.head()


📘 Facebook:


<ipython-input-216-876b0dfd02f0>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_de_acción'] = 'Realizada'
<ipython-input-216-876b0dfd02f0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_contacto'] = df_temp['tipo_contacto']


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,asus,Facebook,Realizada,2
1,aurgi,Facebook,Realizada,2
2,avis,Facebook,Realizada,4
3,beko,Facebook,Realizada,4
4,bosch,Facebook,Realizada,1


In [217]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_facebook['empresa'] = df_facebook['empresa'].str.strip()

df_facebook['empresa'] = (df_facebook['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

**Ejecutar para Twitter**

In [218]:
# Filtramos solo "enviado" por canal
df_twitter  = dfredes[(dfredes['estado_rr_ss_limpio'] == 'realizado') & (dfredes['tipo_contacto'] == 'Twitter')]

# Agregar columnas fijas
for df_temp in [df_twitter]:
    df_temp['tipo_de_acción'] = 'Realizada'
    df_temp['tipo_contacto'] = df_temp['tipo_contacto']

# Agrupar y contar
df_twitter  = df_twitter.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_twitter.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

print("\n Twitter:")
df_twitter.head()


 Twitter:


<ipython-input-218-6cdce953ca22>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_de_acción'] = 'Realizada'
<ipython-input-218-6cdce953ca22>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_contacto'] = df_temp['tipo_contacto']


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,asus,Twitter,Realizada,2
1,aurgi,Twitter,Realizada,2
2,bosch,Twitter,Realizada,1
3,dia fidelización,Twitter,Realizada,1


In [219]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_twitter['empresa'] = df_twitter['empresa'].str.strip()

df_twitter['empresa'] = (df_twitter['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

# Chat y WhatsApp Realizada

In [220]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfchats = pd.read_excel(file_path, sheet_name='Chats')


dfchats.head()

,EMPRESA,tipo_contacto,ESTADO CHAT,FECHA
0,ACER,CHAT WEB,REALIZADO,2025-05-13
1,ASUS,CHAT WEB,REALIZADO,2025-05-13
2,BOSCH,CHAT WEB,REALIZADO,2025-05-13
3,BRUNEAU,CHAT WEB,REALIZADO,2025-05-13
4,DINAHOSTING,CHAT WEB,REALIZADO,2025-05-13


**Corregir las empresas que están mal escritas**

In [221]:
#Pasar a minúscula
dfchats['EMPRESA'] = dfchats['EMPRESA'].str.lower()

# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas'
}

# Aplica los reemplazos
dfchats['EMPRESA'] = dfchats['EMPRESA'].replace(reemplazos)

dfchats['EMPRESA']

,EMPRESA
0,acer
1,asus
2,bosch
3,bruneau
4,dinahosting
5,edenred
6,acer
7,asus
8,bosch
9,bruneau


**Contabilizar la cantidad por canal de realizadas**

In [222]:
# Estandarizamos texto ANTES de cualquier filtro
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.lower()

# Filtramos solo los "realizado"
df_enviados = dfchats[dfchats['estado_chat_limpio'] == 'realizado']

# Definimos categorías válidas
categorias_validas = ['whatsapp', 'chat web']

print(df_enviados['tipo_contacto'].value_counts())


tipo_contacto
chat web    24
Name: count, dtype: int64


**Crear hoja Registro para WhatsApp**

In [223]:
# Estandarizamos texto
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo "enviado" por canal
df_wapp = dfchats[(dfchats['estado_chat_limpio'] == 'realizado') & (dfchats['tipo_contacto'] == 'Whatsapp')]

# Agregar columnas fijas directamente a df_wapp
df_wapp['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
df_wapp = df_wapp.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_wapp.rename(columns={'EMPRESA': 'empresa'}, inplace=True)
df_wapp['tipo_contacto'] = df_wapp['tipo_contacto'].replace('Whatsapp', 'WhatsApp')


# Mostrar resultado
print("📘 WhatsApp:")
df_wapp.head()

📘 WhatsApp:


,empresa,tipo_contacto,tipo_de_acción,cantidad


Pasar las empresas a minúscula

In [224]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_wapp['empresa'] = df_wapp['empresa'].str.strip()

df_wapp['empresa'] = (df_wapp['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

**Crear hoja Chat Web para realizadas**

In [225]:
# Estandarizamos texto
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo "enviado" por canal
df_chat = dfchats[(dfchats['estado_chat_limpio'] == 'realizado') & (dfchats['tipo_contacto'] == 'Chat web')]

# Agregar columnas fijas directamente a df_wapp
df_chat['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
df_chat = df_chat.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_chat.rename(columns={'EMPRESA': 'empresa'}, inplace=True)
#df_chat['tipo_contacto'] = df_chat['tipo_contacto'].replace('Chat', 'Chat web')

# Mostrar resultado
print("📘 Chat:")
df_chat.head()

📘 Chat:


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Chat web,Realizada,4
1,asus,Chat web,Realizada,4
2,bosch,Chat web,Realizada,4
3,bruneau,Chat web,Realizada,4
4,dinahosting,Chat web,Realizada,4


Pasar nombre empresa a minúscula

In [226]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_chat['empresa'] = df_chat['empresa'].str.strip()

df_chat['empresa'] = (df_chat['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

# Agrupar todos los canales Realizada

In [227]:
# === UNIÓN FINAL DE TODOS LOS CANALES ===
df_final_realizada = pd.concat([dftelefono, dfmail, df_facebook, df_twitter, df_chat, df_wapp, dfweb], ignore_index=True)
df_final_realizada = df_final_realizada[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
print(df_final_realizada)

                empresa tipo_contacto tipo_de_acción  cantidad
0                  acer      Telefono      Realizada        48
1               adeslas      Telefono      Realizada        11
2            aeda homes      Telefono      Realizada        22
3           aegon salud      Telefono      Realizada        13
4            aegon vida      Telefono      Realizada         3
..                  ...           ...            ...       ...
182      bosch talleres           Web      Realizada         4
183         bridgestone           Web      Realizada         2
184             bruneau           Web      Realizada         4
185                 bsm           Web      Realizada         3
186  canal de isabel ii           Web      Realizada         4

[187 rows x 4 columns]


In [228]:
recuento_contacto = df_final_realizada.groupby('tipo_contacto')['cantidad'].sum().reset_index()
print("Cantidad total por tipo de contacto:")
print(recuento_contacto)

total_general = df_final_realizada['cantidad'].sum()
print(f"Total general de cantidad: {total_general}")

Cantidad total por tipo de contacto:
     tipo_contacto  cantidad
0         Chat web        24
1         Facebook        40
2  Mail/formulario       483
3         Telefono      1607
4          Twitter         6
5              Web        36
Total general de cantidad: 2196


In [229]:
# Guardar el DataFrame modificado como archivo Excel
df_final_realizada.to_excel('Realizadas.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Realizadas.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ---------------------------------------------------------------------------------------------------------------------------

# **Evaluadas**

# Teléfono

In [230]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluadas.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dftelef = pd.read_excel(file_path, sheet_name='Teléfono')


dftelef.head()

,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C22. IMAGEN CORPORATIVA.COND.NO SE DISCULPA ANTE LAS MOLESTIAS EVIDENCIADAS POR EL CLIENTE DURANTE LA LLAMADA.,C22. IMAGEN CORPORATIVA.OBS,C23. DESPEDIDA,"C23. DESPEDIDA.COND.EL ASESOR SE DESPIDE DEL CLIENTE CON SU NOMBRE, HABIÉNDOSE ASEGURADO DE QUE TODAS SUS DUDAS ESTÉN RESUELTAS.","C23. DESPEDIDA.COND.EL ASESOR SE DESPIDE DEL CLIENTE CON SU NOMBRE, PERO NO SE ASEGURA QUE LE QUEDEN DUDAS POR RESOLVER.","C23. DESPEDIDA.COND.EL ASESOR SE ASEGURA QUE TODAS LAS DUDAS DEL CLIENTE ESTÉN RESUELTAS, PERO NO UTILIZA EL NOMBRE DE ESTE EN LA DESPEDIDA.",C23. DESPEDIDA.COND.EL ASESOR NI SE DESPIDE DEL CLIENTE CON SU NOMBRE NI SE ASEGURA QUE LE QUEDEN DUDAS POR RESOLVER.,C23. DESPEDIDA.COND.NO HA HABIDO DESPEDIDA AL NO HABERSE TRANSFERIDO LA LLAMADA.,C23. DESPEDIDA.COND.NO HA HABIDO DESPEDIDA AL CORTARSE LA TRANSFERENCIA.,C23. DESPEDIDA.OBS
0,101S_1013-OP_11_100_20250506H_15_12,01_HERTZ_01,HERTZ 01,NaN,1013,Hertz Teléfono,Hertz,NaN,10057,2025-05-14,...,NaN,NaN,1.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN
1,101S_1013-OP_11_100_20250507H_12_28,02_HERTZ_02,HERTZ 02,NaN,1013,Hertz Teléfono,Hertz,NaN,10057,2025-05-14,...,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,101S_1025-OP_11_96_20250508H_11_03md$1,01_METROVACESA_05,METROVACESA 05,NaN,1025,Metrovacesa Teléfono,Metrovacesa,NaN,10057,2025-05-22,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,0.0,0.0,NaN
3,101S_1025-OP_11_96_20250508H_12_15md$1,02_METROVACESA_06,METROVACESA 06,NaN,1025,Metrovacesa Teléfono,Metrovacesa,NaN,10057,2025-05-22,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,0.0,0.0,NaN
4,101S_1025-OP_11_96_20250509H_10_50md$1,03_METROVACESA_01,METROVACESA 01,NaN,1025,Metrovacesa Teléfono,Metrovacesa,NaN,10057,2025-05-22,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,0.0,0.0,NaN


In [231]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dftelef['EMPRESA'] = dftelef['EMPRESA'].str.strip()

dftelef['EMPRESA'] = (dftelef['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

In [232]:
# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas'

}

# Aplica los reemplazos
dftelef['EMPRESA'] = dftelef['EMPRESA'].replace(reemplazos)


In [233]:
# Limpiar espacios adelante y atrás en todas las columnas de tipo texto
#dfchatwebE = dfchatwebE.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Asegurar que la columna se llama 'empresa'
dftelef.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Agregar columnas fijas antes del agrupamiento
dftelef['tipo_contacto'] = 'Telefono'
dftelef['tipo_de_acción'] = 'Evaluada'

# Agrupar por empresa, tipo_contacto y tipo_de_acción y contar ocurrencias
dftelef = dftelef.groupby(['empresa', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas
dftelef = dftelef[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
dftelef

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,and ionos,Telefono,Evaluada,1
1,acer,Telefono,Evaluada,38
2,adeslas,Telefono,Evaluada,9
3,aeda homes,Telefono,Evaluada,11
4,asisa,Telefono,Evaluada,3
5,asus,Telefono,Evaluada,8
6,aurgi,Telefono,Evaluada,3
7,avis,Telefono,Evaluada,2
8,beko,Telefono,Evaluada,4
9,bosch,Telefono,Evaluada,7


# Web

In [234]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluadas.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfwebE = pd.read_excel(file_path, sheet_name='Web')


dfwebE.head()

,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C13. FACILIDAD.COND.ES SENCILLO UTILIZAR LOS MENÚS DE NAVEGACIÓN.,C13. FACILIDAD.COND.EN TODA LA NAVEGACIÓN WEB HAY PUBLICIDAD INNECESARIA.,C13. FACILIDAD.COND.NO SE MANTIENEN LOS MISMOS CRITERIOS DE ESTILO EN TODAS LAS PÁGINA FACILITANDO SU USO.,C13. FACILIDAD.COND.NO ES POSIBLE ENCONTRAR CONTENIDOS DE AYUDA SIN DIFICULTAD.,C13. FACILIDAD.COND.NO ES SENCILLO UTILIZAR LOS MENÚS DE NAVEGACIÓN.,C13. FACILIDAD.OBS,C14. DIVERSIDAD FUNCIONAL,C14. DIVERSIDAD FUNCIONAL.COND.LA PÁGINA WEB SÍ TIENE OPCIÓN DE ACCESO PARA PERSONAS CON DISCAPACIDAD VISUAL.,C14. DIVERSIDAD FUNCIONAL.COND.LA PÁGINA WEB NO TIENE OPCIÓN DE ACCESO PARA PERSONAS CON DISCAPACIDAD VISUAL.,C14. DIVERSIDAD FUNCIONAL.OBS
0,101S_1066-OP_11_98_20250513H_16_04,01_BSM_01,BSM 01,NaN,1066,BSM Web,BSM,NaN,10059,2025-05-20,...,0,NaN,NaN,NaN,NaN,NaN,2,1,NaN,NaN
1,101S_1066-OP_11_98_20250514H_10_49,02_BSM_11,BSM 11,NaN,1066,BSM Web,BSM,NaN,10059,2025-05-20,...,0,NaN,NaN,NaN,NaN,NaN,2,1,NaN,NaN
2,101S_1066-OP_11_98_20250515H_11_52,03_BSM_14,BSM 14,NaN,1066,BSM Web,BSM,NaN,10059,2025-05-20,...,0,NaN,NaN,NaN,NaN,NaN,2,1,NaN,NaN
3,101S_106-OP_11_98_20250513H_11_24md$1,01_ACER_02,Acer 2,NaN,106,Acer Web,Acer,NaN,10059,2025-05-15,...,0,NaN,NaN,NaN,NaN,NaN,2,1,NaN,NaN
4,101S_106-OP_11_98_20250514H_09_12,02_ACER_08,Acer 8,NaN,106,Acer Web,Acer,NaN,10059,2025-05-15,...,0,NaN,NaN,NaN,NaN,NaN,2,1,NaN,NaN


In [235]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfwebE['EMPRESA'] = dfwebE['EMPRESA'].str.strip()

dfwebE['EMPRESA'] = (dfwebE['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

In [236]:
# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas'
}

# Aplica los reemplazos
dfwebE['EMPRESA'] = dfwebE['EMPRESA'].replace(reemplazos)

In [237]:
# Asegurar que la columna se llama 'empresa'
dfwebE.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Agregar columnas fijas antes del agrupamiento
dfwebE['tipo_contacto'] = 'Web'
dfwebE['tipo_de_acción'] = 'Evaluada'

# Agrupar por empresa, tipo_contacto y tipo_de_acción y contar ocurrencias
dfwebE = dfwebE.groupby(['empresa', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas
dfwebE = dfwebE[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
dfwebE

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Web,Evaluada,3
1,adeslas,Web,Evaluada,2
2,aeda homes,Web,Evaluada,3
3,asisa,Web,Evaluada,2
4,asus,Web,Evaluada,2
5,aurgi,Web,Evaluada,2
6,avis,Web,Evaluada,2
7,beko,Web,Evaluada,1
8,bosch,Web,Evaluada,1
9,bosch talleres,Web,Evaluada,4


# Facebook y Twitter

In [238]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluadas.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfredesE = pd.read_excel(file_path, sheet_name='Redes')


dfredesE.head()

,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C14. HABILIDAD.COND.NO EXISTE OPCIÓN DE ADJUNTAR MATERIAL MULTIMEDIA PARA COMPLEMENTAR LA CONSULTA.,C14. HABILIDAD.OBS,C15. SOLUCIÓN AL PRIMER CONTACTO (FCR),C15. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y NO ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.,C15. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.NO SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.,C15. SOLUCIÓN AL PRIMER CONTACTO (FCR).OBS,C7. FORMATO DE LA RESPUESTA.COND.LA RESPUESTA OCUPA MENOS DE 1 TWEET/6 LÍNEAS DE MD EN TWITTER.,C7. FORMATO DE LA RESPUESTA.COND.LA RESPUESTA OCUPA MÁS DE 1 TWEET/6 LÍNEAS DE MD EN TWITTER.,C11. DESPEDIDA.COND.EL ASESOR SÍ REALIZA UNA DESPEDIDA CORDIAL PARA DESPEDIR LA CONVERSACIÓN EN TWITTER.,C11. DESPEDIDA.COND.EL ASESOR NO REALIZA UNA DESPEDIDA CORDIAL PARA DESPEDIR LA CONVERSACIÓN EN TWITTER.
0,101S_1108-OP_11_98_20250513H_13_04,01_AURGI_01,Aurgi 01,NaN,1108,Aurgi Facebook,Aurgi,NaN,10061,2025-05-19,...,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,101S_1108-OP_11_98_20250514H_15_04,02_AURGI_02,Aurgi 02,NaN,1108,Aurgi Facebook,Aurgi,NaN,10061,2025-05-19,...,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,101S_1109-OP_11_98_20250515H_09_39,03_AURGI_03,Aurgi 03,NaN,1109,Aurgi Twitter,Aurgi,NaN,10060,2025-05-19,...,NaN,NaN,2.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0
3,101S_1109-OP_11_98_20250516H_09_20,04_AURGI_04,Aurgi 04,NaN,1109,Aurgi Twitter,Aurgi,NaN,10060,2025-05-19,...,NaN,NaN,2.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0
4,101S_1113-OP_11_98_20250513H_16_00,01_AVIS_01,Avis 01,NaN,1113,Avis Facebook,Avis,NaN,10061,2025-05-19,...,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [239]:
#Pasar a minúscula
dfredesE['EMPRESA'] = dfredesE['EMPRESA'].str.lower()

# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas'
}

# Aplica los reemplazos
dfredesE['EMPRESA'] = dfredesE['EMPRESA'].replace(reemplazos)

dfredesE['EMPRESA']

,EMPRESA
0,aurgi
1,aurgi
2,aurgi
3,aurgi
4,avis
5,avis
6,avis
7,avis
8,continental
9,continental


In [240]:
# --- PASO 1: Asegurar minúsculas en columnas relevantes ---
dfredesE['SERVICIO'] = dfredesE['SERVICIO'].str.lower()
dfredesE['EMPRESA'] = (dfredesE['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

# --- PASO 2: Definir palabras clave válidas y reemplazos ---
servicios_validos = [
    'twitter','facebook', 'canal social media facebook', 'social media facebook'
]

reemplazos_servicio = {
    'teléfono': 'Telefono',
    'llamadas': 'Telefono',
    'telefónico': 'Telefono',
    'canal telefónico': 'Telefono',
    'canal web':'Web',
    'canal social media facebook':'Facebook',
    'social media facebook': 'Facebook',
    'mail/formulario':'Mail/formulario',
    'web': 'Web',
    'facebook':'Facebook',
    'twitter':'Twitter',
    'chat web': 'Chat web',
    'whatsapp': 'WhatsApp',
    'webs':'Web'
}

# --- PASO 3: Función para limpiar SERVICIO ---
def limpiar_servicio(servicio):
    for palabra in servicios_validos:
        if palabra in servicio:
            return reemplazos_servicio.get(palabra, palabra)
    return None

# --- PASO 4: Aplicar limpieza servicio ---
dfredesE['SERVICIO'] = dfredesE['SERVICIO'].apply(limpiar_servicio)

print(dfredesE['SERVICIO'])

0     Facebook
1     Facebook
2      Twitter
3      Twitter
4     Facebook
5     Facebook
6     Facebook
7     Facebook
8     Facebook
9     Facebook
10    Facebook
11    Facebook
12    Facebook
13    Facebook
14    Facebook
15     Twitter
16    Facebook
17    Facebook
18    Facebook
19    Facebook
20    Facebook
21    Facebook
22    Facebook
23    Facebook
24    Facebook
25    Facebook
26    Facebook
27    Facebook
28    Facebook
29    Facebook
30    Facebook
31    Facebook
32    Facebook
33    Facebook
34    Facebook
35    Facebook
36    Facebook
37    Facebook
Name: SERVICIO, dtype: object


In [241]:
# Filtramos solo "enviado" por canal
df_facebookE = dfredesE[(dfredesE['SERVICIO'] == 'Facebook')]

# Agregar columnas fijas
for df_temp in [df_facebookE]:
    df_temp['tipo_de_acción'] = 'Evaluada'
    df_temp['tipo_contacto'] = df_temp['SERVICIO']

# Agrupar y contar
df_facebookE = df_facebookE.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_facebookE.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar ambos resultados
print("📘 Facebook:")
df_facebookE.head()

📘 Facebook:


<ipython-input-241-dc306191d786>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_de_acción'] = 'Evaluada'
<ipython-input-241-dc306191d786>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_contacto'] = df_temp['SERVICIO']


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,asus,Facebook,Evaluada,2
1,aurgi,Facebook,Evaluada,2
2,avis,Facebook,Evaluada,4
3,bosch talleres,Facebook,Evaluada,4
4,bridgestone,Facebook,Evaluada,4


**TWITTER**

In [242]:
# Filtramos solo "enviado" por canal
df_twitterE = dfredesE[(dfredesE['SERVICIO'] == 'Twitter')]

# Agregar columnas fijas
for df_temp in [df_twitterE]:
    df_temp['tipo_de_acción'] = 'Evaluada'
    df_temp['tipo_contacto'] = df_temp['SERVICIO']

# Agrupar y contar
df_twitterE = df_twitterE.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_twitterE.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar ambos resultados
print("📘 Twitter:")
df_twitterE.head()

📘 Twitter:


<ipython-input-242-c34eacc6f830>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_de_acción'] = 'Evaluada'
<ipython-input-242-c34eacc6f830>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_contacto'] = df_temp['SERVICIO']


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,aurgi,Twitter,Evaluada,2
1,dia fidelización,Twitter,Evaluada,1


# Chat

In [243]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluadas.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfchatwebE = pd.read_excel(file_path, sheet_name='Chat web')


dfchatwebE.head()

,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C16. HISTÓRICO DE LA CONVERSACIÓN,C16. HISTÓRICO DE LA CONVERSACIÓN.COND.SÍ SE ENVÍA UN MENSAJE AL FINAL DÓNDE SE INDIQUE SE LA CONVERSACIÓN SE ENVIARÁ POR CORREO ELECTRÓNICO.,C16. HISTÓRICO DE LA CONVERSACIÓN.COND.SÍ EXISTE UNA OPCIÓN EN EL CHAT QUE PERMITE PODER GUARDAR LA CONVERSACIÓN MANTENIDA CON EL ASESOR.,C16. HISTÓRICO DE LA CONVERSACIÓN.COND.NO SE ENVÍA UN MENSAJE AL FINAL DÓNDE SE INDIQUE SE LA CONVERSACIÓN SE ENVIARÁ POR CORREO ELECTRÓNICO.,C16. HISTÓRICO DE LA CONVERSACIÓN.COND.NO EXISTE UNA OPCIÓN EN EL CHAT QUE PERMITE PODER GUARDAR LA CONVERSACIÓN MANTENIDA CON EL ASESOR.,C16. HISTÓRICO DE LA CONVERSACIÓN.OBS,C17. SOLUCIÓN AL PRIMER CONTACTO (FCR),C17. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO YNO ES NECESARIO REALIZAR UN CONTACTO POSTERIOR,C17. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.NO SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y ES NECESARIO REALIZAR UN CONTACTO POSTERIOR,C17. SOLUCIÓN AL PRIMER CONTACTO (FCR).OBS
0,101S_1124-OP_11_98_20250513H_16_50,01_BOSCH_01,Bosch 01,NaN,1124,Bosch Chat Web,Bosch,NaN,10062,2025-05-20,...,0.0,NaN,NaN,1.0,0.0,NaN,2.0,1.0,NaN,NaN
1,101S_1124-OP_11_98_20250514H_12_48,02_BOSCH_02,Bosch 02,NaN,1124,Bosch Chat Web,Bosch,NaN,10062,2025-05-20,...,0.0,NaN,NaN,1.0,0.0,NaN,2.0,1.0,NaN,NaN
2,101S_1124-OP_11_98_20250515H_14_35,03_BOSCH_03,Bosch 03,NaN,1124,Bosch Chat Web,Bosch,NaN,10062,2025-05-21,...,0.0,NaN,NaN,1.0,0.0,NaN,0.0,NaN,1.0,NaN
3,101S_1124-OP_11_98_20250516H_13_21,04_BOSCH_04,Bosch 04,NaN,1124,Bosch Chat Web,Bosch,NaN,10062,2025-05-21,...,0.0,NaN,NaN,0.0,1.0,NaN,2.0,1.0,NaN,NaN
4,101S_553-OP_11_98_20250513H_12_15,01_ACER_01,Acer 1,NaN,553,Acer Chat Virtual,Acer,NaN,10062,2025-05-19,...,2.0,1.0,0.0,NaN,NaN,NaN,2.0,1.0,NaN,NaN


In [244]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfchatwebE['EMPRESA'] = dfchatwebE['EMPRESA'].str.strip()

dfchatwebE['EMPRESA'] = (dfchatwebE['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

In [245]:
# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas'
}

# Aplica los reemplazos
dfchatwebE['EMPRESA'] = dfchatwebE['EMPRESA'].replace(reemplazos)

dfchatwebE['EMPRESA']
print(dfchatwebE[dfchatwebE['EMPRESA'] == 'canal de isabel ii'])

Empty DataFrame
Columns: [ID, IDEVENTCLIENTE, EQUIPO, CIP, ID SERVICIO, SERVICIO, EMPRESA, DISPOSITIVO, MEDIDOR, FECHA EVALUACIÓN, HORAEVALUACION, VALORACIÓN, IDOPERADOR, AGENTE, IDEVALUADOR, EVALUADOR, IDESPECIALISTA, RESPUESTA, DURACIÓN, FECHA, HORA, MOTIVO, ALCANCE, CATEGORIA1, CATEGORIA2, CATEGORIA3, CATEGORIA4, GRABACIÓN, ARCHIVO_ADJUNTO, ARCHIVO_ADJUNTO2, ARCHIVO_ADJUNTO3, ARCHIVO_ADJUNTO4, ARCHIVO_ADJUNTO5, C1. VISIBILIDAD, C1. VISIBILIDAD.COND.SE NECESITAN 4 O MENOS CLICKS PARA ACCEDER AL CHAT WEB., C1. VISIBILIDAD.COND.SE NECESITAN ENTRE 5 Y 6 CLICKS PARA ACCEDER AL CHAT WEB., C1. VISIBILIDAD.COND.SE NECESITAN 7 O MÁS CLICKS PARA ACCEDER AL CHAT WEB., C1. VISIBILIDAD.OBS, C2. TIEMPO DE ESPERA, C2. TIEMPO DE ESPERA.COND.EL ASESOR ATIENDE AL CLIENTE EN MENOS DE 30 SEGUNDOS., C2. TIEMPO DE ESPERA.COND.EL ASESOR ATIENDE AL CLIENTE ENTRE 30 SEGUNDOS Y 1 MINUTO 30 SEGUNDOS., C2. TIEMPO DE ESPERA.COND.EL ASESOR ATIENDE AL CLIENTE ENTRE 1 MINUTO Y 30 SEGUNDOS Y 3 MINUTOS., C2. TIEMPO 

In [246]:
# Limpiar espacios adelante y atrás en todas las columnas de tipo texto
#dfchatwebE = dfchatwebE.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Asegurar que la columna se llama 'empresa'
dfchatwebE.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Agregar columnas fijas antes del agrupamiento
dfchatwebE['tipo_contacto'] = 'Chat'
dfchatwebE['tipo_de_acción'] = 'Evaluada'

# Agrupar por empresa, tipo_contacto y tipo_de_acción y contar ocurrencias
dfchatwebE = dfchatwebE.groupby(['empresa', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas
dfchatwebE = dfchatwebE[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
dfchatwebE


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Chat,Evaluada,4
1,asus,Chat,Evaluada,2
2,bosch,Chat,Evaluada,4
3,bruneau,Chat,Evaluada,4
4,dinahosting,Chat,Evaluada,4


# **Agrupar por EVALUADAS**

In [247]:
# === UNIÓN FINAL DE TODOS LOS CANALES ===
df_final_evaluada = pd.concat([dftelef, df_facebookE, df_twitterE, dfchatwebE, dfwebE], ignore_index=True)
df_final_evaluada = df_final_evaluada[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
print(df_final_evaluada)

               empresa tipo_contacto tipo_de_acción  cantidad
0           and  ionos      Telefono       Evaluada         1
1                 acer      Telefono       Evaluada        38
2              adeslas      Telefono       Evaluada         9
3           aeda homes      Telefono       Evaluada        11
4                asisa      Telefono       Evaluada         3
..                 ...           ...            ...       ...
75      bosch talleres           Web       Evaluada         4
76         bridgestone           Web       Evaluada         2
77             bruneau           Web       Evaluada         4
78                 bsm           Web       Evaluada         3
79  canal de isabel ii           Web       Evaluada         4

[80 rows x 4 columns]


In [248]:
recuento_contacto = df_final_evaluada.groupby('tipo_contacto')['cantidad'].sum().reset_index()
print("Cantidad total por tipo de contacto:")
print(recuento_contacto)

total_general = df_final_evaluada['cantidad'].sum()
print(f"Total general de cantidad: {total_general}")

Cantidad total por tipo de contacto:
  tipo_contacto  cantidad
0          Chat        18
1      Facebook        35
2      Telefono       435
3       Twitter         3
4           Web        35
Total general de cantidad: 526


In [249]:
# Guardar el DataFrame modificado como archivo Excel
df_final_evaluada.to_excel('Registro.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Registro.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ---------------------------------------------------------------------------------------------------------------------------------

# **Crear hoja Recuento por TIPO DE CONTACTO**

In [250]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Resumen Planificación.xlsx'

# Cargar el archivo Excel
df2 = pd.read_excel(file_path)

# Mostrar las primeras filas del archivo
df2.head()

,Empresa,Categoría,Inicio,Fin,BBDD,Observaciones BBDD,Compra,Fecha Informe,Telefono,Mail/formulario,...,Twitter,Facebook,Chat,WhatsApp,Confirmación,Informe,Alta en eAlicia,Escenarios C3,Emisión (horas),Evaluación (horas)
0,Acer,Hosting,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,135,55,...,6.0,6.0,12.0,0.0,NaN,Sí (Inglés),NaN,NaN,19.65,29.475
1,Adeslas,Hosting,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,45,18,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,19.65,29.475
2,ADT,ASCENSORES,2025-05-19,2025-07-04,NaN,NaN,NaN,NaN,45,18,...,0.0,0.0,0.0,0.0,NaN,Sí,NaN,NaN,17.25,25.875
3,Aeda Homes,ASCENSORES,2025-05-12,2025-07-04,NaN,NaN,NaN,NaN,68,27,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,18.65,27.975
4,Aegon Salud,Sistemas de Seguridad,2025-05-05,2025-06-27,Sí,Datos reales y CP qué debemos decir en los cli...,NaN,NaN,135,35,...,0.0,0.0,0.0,0.0,NaN,Sí,NaN,NaN,18.65,27.975


**Unir los dos excel, obteniendo evaluada, realizada y total**

In [251]:
# --- Paso 1: Unir evaluadas y realizadas ---
df_combined = pd.concat([df_final_evaluada, df_final_realizada], ignore_index=True)

df_combined

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,and ionos,Telefono,Evaluada,1
1,acer,Telefono,Evaluada,38
2,adeslas,Telefono,Evaluada,9
3,aeda homes,Telefono,Evaluada,11
4,asisa,Telefono,Evaluada,3
...,...,...,...,...
262,bosch talleres,Web,Realizada,4
263,bridgestone,Web,Realizada,2
264,bruneau,Web,Realizada,4
265,bsm,Web,Realizada,3


In [252]:
import pandas as pd

# --- Normalización mínima: solo minúsculas y espacios ---
df_combined['empresa'] = df_combined['empresa'].str.lower().str.strip()
df_combined['tipo_contacto'] = df_combined['tipo_contacto'].str.strip()

df2['Empresa'] = df2['Empresa'].str.lower().str.strip()
canales = ['Telefono', 'Mail/formulario', 'Web', 'Facebook', 'Twitter', 'Chat', 'WhatsApp']

# --- Paso 1: Pivotear df_combined por empresa y canal ---
df_pivot = df_combined.pivot_table(
    index=['empresa', 'tipo_contacto'],
    columns='tipo_de_acción',
    values='cantidad',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Asegurarse de que columnas Evaluada y Realizada estén presentes
for col in ['Evaluada', 'Realizada']:
    if col not in df_pivot.columns:
        df_pivot[col] = 0

# Ordenar columnas
df_pivot = df_pivot[['empresa', 'tipo_contacto', 'Evaluada', 'Realizada']]

# --- Paso 2: Pasar df2 a formato largo ---
df2_melt = df2.melt(
    id_vars=['Empresa', 'Inicio', 'Fin'],
    value_vars=canales,
    var_name='tipo_contacto',
    value_name='Total'
)

df2_melt.rename(columns={'Empresa': 'empresa'}, inplace=True)
df2_melt['tipo_contacto'] = df2_melt['tipo_contacto'].str.strip()

# --- Paso 3: Unir los datos sin alterar nombres originales ---
df_merged = pd.merge(
    df2_melt,
    df_pivot,
    on=['empresa', 'tipo_contacto'],
    how='left'
)

# Rellenar NaN solo en columnas numéricas
df_merged[['Evaluada', 'Realizada', 'Total']] = df_merged[['Evaluada', 'Realizada', 'Total']].fillna(0)

# --- Paso 4: Pivotear a formato final (una fila por empresa con columnas por canal) ---
df_final = df_merged.pivot_table(
    index=['empresa', 'Inicio', 'Fin'],
    columns='tipo_contacto',
    values=['Evaluada', 'Realizada', 'Total'],
    aggfunc='sum',
    fill_value=0
)

# Aplanar columnas multiíndice
df_final.columns = [f"{canal}_{accion}" for accion, canal in df_final.columns]
df_final = df_final.reset_index()

# --- Paso 5: Unir con todas las empresas de df2 para no perder ninguna ---
df_base_empresas = df2[['Empresa', 'Inicio', 'Fin']].copy()
df_base_empresas.rename(columns={'Empresa': 'empresa'}, inplace=True)
df_base_empresas['empresa'] = df_base_empresas['empresa'].str.lower().str.strip()

# Unión externa para mantener empresas sin datos de acciones
df_final_completo = pd.merge(
    df_base_empresas,
    df_final,
    on=['empresa', 'Inicio', 'Fin'],
    how='left'
)

# Rellenar posibles NaN por empresas sin datos de acciones
df_final_completo = df_final_completo.fillna(0)

# --- Paso 6: Verificación de consistencia ---
empresas_originales = set(df2['Empresa'].str.lower().str.strip())
empresas_final = set(df_final_completo['empresa'])
empresas_faltantes = empresas_originales - empresas_final

if empresas_faltantes:
    print(f"⚠️ Empresas faltantes en el resultado final: {sorted(empresas_faltantes)}")
else:
    print("✅ Todas las empresas de df2 están presentes en df_final.")

# Resultado final
df_final = df_final_completo

df_final



✅ Todas las empresas de df2 están presentes en df_final.


,empresa,Inicio,Fin,Chat_Evaluada,Facebook_Evaluada,Mail/formulario_Evaluada,Telefono_Evaluada,Twitter_Evaluada,Web_Evaluada,WhatsApp_Evaluada,...,Twitter_Realizada,Web_Realizada,WhatsApp_Realizada,Chat_Total,Facebook_Total,Mail/formulario_Total,Telefono_Total,Twitter_Total,Web_Total,WhatsApp_Total
0,acer,2025-05-05,2025-06-27,4.0,0.0,0.0,38.0,0.0,3.0,0.0,...,0.0,4.0,0.0,12.0,6.0,55.0,135.0,6.0,6.0,0.0
1,adeslas,2025-05-05,2025-06-27,0.0,0.0,0.0,9.0,0.0,2.0,0.0,...,0.0,2.0,0.0,0.0,0.0,18.0,45.0,0.0,2.0,0.0
2,adt,2025-05-19,2025-07-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,18.0,45.0,0.0,2.0,0.0
3,aeda homes,2025-05-12,2025-07-04,0.0,0.0,0.0,11.0,0.0,3.0,0.0,...,0.0,3.0,0.0,0.0,0.0,27.0,68.0,0.0,3.0,0.0
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,35.0,135.0,0.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,volotea,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,55.0,135.0,0.0,6.0,0.0
97,vueling,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19.0,45.0,0.0,2.0,0.0
98,whirlpool,2025-06-02,2025-06-27,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,14.0,34.0,2.0,2.0,0.0
99,xiaomi,2025-06-02,2025-06-27,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,19.0,45.0,4.0,2.0,2.0


In [253]:
#Empresas sin fecha de inicio y fin
#print("📘 Empresas sin fecha de inicio y fin, a reemplazar a mano:")
#display(df_final_sin_fecha[['empresa'] + [col for col in df_final_sin_fecha.columns if col in ['Inicio', 'Fin']]])


**Ordenar columnas para que esten juntos por canal**

In [254]:
# Paso final: ordenar columnas agrupadas por canal
orden_final = ['empresa', 'Inicio', 'Fin']
tipos = ['Realizada', 'Evaluada', 'Total']
for canal in canales:
    for tipo in tipos:
        col = f"{canal}_{tipo}"
        if col in df_final.columns:
            orden_final.append(col)

# Reordenar el DataFrame
df_final = df_final[orden_final]

df_final

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Facebook_Total,Twitter_Realizada,Twitter_Evaluada,Twitter_Total,Chat_Realizada,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total
0,acer,2025-05-05,2025-06-27,48.0,38.0,135.0,12.0,0.0,55.0,4.0,...,6.0,0.0,0.0,6.0,0.0,4.0,12.0,0.0,0.0,0.0
1,adeslas,2025-05-05,2025-06-27,11.0,9.0,45.0,4.0,0.0,18.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,adt,2025-05-19,2025-07-04,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,aeda homes,2025-05-12,2025-07-04,22.0,11.0,68.0,4.0,0.0,27.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon salud,2025-05-05,2025-06-27,13.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,volotea,2025-06-02,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,vueling,2025-06-02,2025-06-27,0.0,0.0,45.0,0.0,0.0,19.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,whirlpool,2025-06-02,2025-06-27,13.0,2.0,34.0,4.0,0.0,14.0,0.0,...,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
99,xiaomi,2025-06-02,2025-06-27,12.0,1.0,45.0,2.0,0.0,19.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,2.0


**--Generar una copia del df_final hasta aquí para luego trabajar el acumulativo--**

In [255]:
# Crear una copia profunda del DataFrame (independiente de la original) para trabajar con el acumulativo a posteriori
#df_copy = df_final.copy()

#print("Copia modificada")
#print(df_copy.head())


**Obtener Total_general, total_evaluadas y total_realizadas**

In [256]:
# Identificamos columnas por tipo
cols_realizada = [col for col in df_final.columns if col.endswith('_Realizada')]
cols_evaluada = [col for col in df_final.columns if col.endswith('_Evaluada')]
cols_total = [col for col in df_final.columns if col.endswith('_Total')]

# Creamos las columnas de suma total
df_final['Total_realizadas'] = df_final[cols_realizada].sum(axis=1)
df_final['Total_evaluadas'] = df_final[cols_evaluada].sum(axis=1)
df_final['Total_general'] = df_final[cols_total].sum(axis=1)

df_final.head()

<ipython-input-256-e27012156d36>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Total_realizadas'] = df_final[cols_realizada].sum(axis=1)
<ipython-input-256-e27012156d36>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Total_evaluadas'] = df_final[cols_evaluada].sum(axis=1)
<ipython-input-256-e27012156d36>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Twitter_Total,Chat_Realizada,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general
0,acer,2025-05-05,2025-06-27,48.0,38.0,135.0,12.0,0.0,55.0,4.0,...,6.0,0.0,4.0,12.0,0.0,0.0,0.0,64.0,45.0,220.0
1,adeslas,2025-05-05,2025-06-27,11.0,9.0,45.0,4.0,0.0,18.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,11.0,65.0
2,adt,2025-05-19,2025-07-04,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65.0
3,aeda homes,2025-05-12,2025-07-04,22.0,11.0,68.0,4.0,0.0,27.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,14.0,98.0
4,aegon salud,2025-05-05,2025-06-27,13.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,176.0


**Obtener los dias laborables restantes**

In [257]:
import numpy as np
from datetime import datetime
from workalendar.europe.spain import Catalonia  # o Spain, SpainCatalonia, etc.

# Asegurar que la columna Fin está en formato datetime
df_final['Fin'] = pd.to_datetime(df_final['Fin'])

# Fecha de hoy
hoy = np.datetime64(datetime.today().date(), 'D')

# Convertir 'Fin' a datetime64[D] para que funcione con busday_count
fechas_fin = df_final['Fin'].values.astype('datetime64[D]')

# Obtener feriados nacionales + Madrid del año actual y siguiente
cal = Catalonia()
año_actual = datetime.today().year
feriados = cal.holidays(año_actual) + cal.holidays(año_actual + 1)
feriados_np = np.array([np.datetime64(d[0], 'D') for d in feriados])

# Calcular días hábiles restantes excluyendo feriados
df_final['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)

# Visualizar
df_final.head()


<ipython-input-257-3f456854cbaf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Fin'] = pd.to_datetime(df_final['Fin'])
<ipython-input-257-3f456854cbaf>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)


,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Chat_Realizada,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general,Dias_laborables_restantes
0,acer,2025-05-05,2025-06-27,48.0,38.0,135.0,12.0,0.0,55.0,4.0,...,0.0,4.0,12.0,0.0,0.0,0.0,64.0,45.0,220.0,23
1,adeslas,2025-05-05,2025-06-27,11.0,9.0,45.0,4.0,0.0,18.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,17.0,11.0,65.0,23
2,adt,2025-05-19,2025-07-04,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65.0,28
3,aeda homes,2025-05-12,2025-07-04,22.0,11.0,68.0,4.0,0.0,27.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,29.0,14.0,98.0,28
4,aegon salud,2025-05-05,2025-06-27,13.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,176.0,23


**Obtener columnas por evaluar y por realizar según el total general**

In [258]:
df_final['Por_evaluar'] = df_final['Total_general'] - df_final['Total_evaluadas']
df_final['Por_realizar'] = df_final['Total_general'] - df_final['Total_realizadas']

df_final.head()

<ipython-input-258-257f9cce6d9f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Por_evaluar'] = df_final['Total_general'] - df_final['Total_evaluadas']
<ipython-input-258-257f9cce6d9f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Por_realizar'] = df_final['Total_general'] - df_final['Total_realizadas']


,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general,Dias_laborables_restantes,Por_evaluar,Por_realizar
0,acer,2025-05-05,2025-06-27,48.0,38.0,135.0,12.0,0.0,55.0,4.0,...,12.0,0.0,0.0,0.0,64.0,45.0,220.0,23,175.0,156.0
1,adeslas,2025-05-05,2025-06-27,11.0,9.0,45.0,4.0,0.0,18.0,2.0,...,0.0,0.0,0.0,0.0,17.0,11.0,65.0,23,54.0,48.0
2,adt,2025-05-19,2025-07-04,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,65.0,28,65.0,65.0
3,aeda homes,2025-05-12,2025-07-04,22.0,11.0,68.0,4.0,0.0,27.0,3.0,...,0.0,0.0,0.0,0.0,29.0,14.0,98.0,28,84.0,69.0
4,aegon salud,2025-05-05,2025-06-27,13.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,0.0,0.0,13.0,0.0,176.0,23,176.0,163.0


**Calcular la cantidad de a realizar y evaluar según los días laborables restantes**

In [259]:
# Asegurarse de evitar división por cero o días negativos
df_final['Dias_laborables_restantes'] = df_final['Dias_laborables_restantes'].clip(lower=1)

# Calcular valores por día
df_final['A_realizar_por_dia'] = ((df_final['Total_general'] - df_final['Total_realizadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)
df_final['A_evaluar_por_dia'] = ((df_final['Total_general'] - df_final['Total_evaluadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)

df_final.head()

<ipython-input-259-244ce7dcf6b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Dias_laborables_restantes'] = df_final['Dias_laborables_restantes'].clip(lower=1)
<ipython-input-259-244ce7dcf6b5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['A_realizar_por_dia'] = ((df_final['Total_general'] - df_final['Total_realizadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)
<ipython-input-259-244ce7dcf6b5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general,Dias_laborables_restantes,Por_evaluar,Por_realizar,A_realizar_por_dia,A_evaluar_por_dia
0,acer,2025-05-05,2025-06-27,48.0,38.0,135.0,12.0,0.0,55.0,4.0,...,0.0,0.0,64.0,45.0,220.0,23,175.0,156.0,7,8
1,adeslas,2025-05-05,2025-06-27,11.0,9.0,45.0,4.0,0.0,18.0,2.0,...,0.0,0.0,17.0,11.0,65.0,23,54.0,48.0,2,2
2,adt,2025-05-19,2025-07-04,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,0.0,0.0,65.0,28,65.0,65.0,2,2
3,aeda homes,2025-05-12,2025-07-04,22.0,11.0,68.0,4.0,0.0,27.0,3.0,...,0.0,0.0,29.0,14.0,98.0,28,84.0,69.0,2,3
4,aegon salud,2025-05-05,2025-06-27,13.0,0.0,135.0,0.0,0.0,35.0,0.0,...,0.0,0.0,13.0,0.0,176.0,23,176.0,163.0,7,8


**Extraer el archivo final**

In [260]:
# Guardar el DataFrame modificado como archivo Excel
df_final.to_excel('Recuento por TIPO DE CONTACTO.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Recuento por TIPO DE CONTACTO.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ----------------------------------------------------------------------------------------------

# **Extraer el canal de la columna SERVICIO para Evaluadas de eAlicia**

Ejecutar solo las dos primeras celdas, la última es para descargar el excel

In [36]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluaciones prueba.xlsx'

# Cargar el archivo Excel
df = pd.read_excel(file_path)

# Mostrar las primeras filas del archivo
df.head()


,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C13. CALIDAD DE LA RESPUESTA.COND.LA RESPUESTA PROPORCIONADA DURANTE LA CONVERSACIÓN DE WHATSAPP ES FALSA. MUESTRA NULO CONOCIMIENTO (ENTENDIMIENTO) DE LA CONSULTA O PRODUCTO.,C13. CALIDAD DE LA RESPUESTA.COND.EL CHAT DE WHATSAPP NO HA SIDO ATENDIDO TRAS PASARSE EL PLAZO DE 5 MINUTOS Y NO RECIBIR RESPUESTA POR PARTE DE UN ASESOR.,C14. FUNCIONALIDAD.COND.EL LINK FACILITADO POR EL ASESOR LLEVA A UNA PÁGINA CON LA SOLUCIÓN A LA CONSULTA PLANTEADA POR EL CLIENTE.,C14. FUNCIONALIDAD.COND.EL LINK FACILITADO POR EL ASESOR LLEVA A UNA PÁGINA DIFERENTE A LO CONSULTA DESCRITA POR EL CLIENTE.,C15. GESTIÓN CORRECTA DE LA SOLICITUD,C15. GESTIÓN CORRECTA DE LA SOLICITUD.COND.EL ASESOR SÍ DERIVA AL CANAL CORRECTO DONDE EL CLIENTE PUEDE SOLVENTAR LA CONSULTA.,C15. GESTIÓN CORRECTA DE LA SOLICITUD.COND.EL ASESOR NO DERIVA AL CANAL CORRECTO DONDE EL CLIENTE PUEDE SOLVENTAR LA CONSULTA.,C16. SOLUCIÓN AL PRIMER CONTACTO (FCR),C16. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y NO ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.,C16. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.NO SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.
0,101S_104-OP_11_100_20250506H_14_49,01_ACER_01,Acer 1,NaN,104,Acer Teléfono,Acer,NaN,10057,2025-05-09,...,,,,,,,,,,
1,101S_104-OP_11_100_20250506H_15_28,02_ACER_04,Acer 4,NaN,104,Acer Teléfono,Acer,NaN,10057,2025-05-09,...,,,,,,,,,,
2,101S_104-OP_11_100_20250507H_10_28,03_ACER_05,Acer 5,NaN,104,Acer Teléfono,Acer,NaN,10057,2025-05-09,...,,,,,,,,,,
3,101S_104-OP_11_100_20250507H_10_52,04_ACER_03,Acer 3,NaN,104,Acer Teléfono,Acer,NaN,10057,2025-05-09,...,,,,,,,,,,
4,101S_104-OP_11_100_20250507H_11_48,05_ACER_06,Acer 6,NaN,104,Acer Teléfono,Acer,NaN,10057,2025-05-09,...,,,,,,,,,,


**De la columna servicio extraer SERVICIO**

In [37]:
# --- PASO 1: Asegurar minúsculas en columnas relevantes ---
df['SERVICIO'] = df['SERVICIO'].str.lower()
df['EMPRESA'] = (df['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

# --- PASO 2: Definir palabras clave válidas y reemplazos ---
servicios_validos = [
    'teléfono', 'llamadas', 'mail/formulario', 'web', 'twitter',
    'facebook', 'chat web', 'whatsapp', 'canal telefónico', 'telefónico',
    'canal web', 'canal social media facebook', 'social media facebook','webs'
]

reemplazos_servicio = {
    'teléfono': 'Telefono',
    'llamadas': 'Telefono',
    'telefónico': 'Telefono',
    'canal telefónico': 'Telefono',
    'canal web':'Web',
    'canal social media facebook':'Facebook',
    'social media facebook': 'Facebook',
    'mail/formulario':'Mail/formulario',
    'web': 'Web',
    'facebook':'Facebook',
    'twitter':'Twitter',
    'chat web': 'Chat web',
    'whatsapp': 'WhatsApp',
    'webs':'Web'
}

# --- PASO 3: Función para limpiar SERVICIO ---
def limpiar_servicio(servicio):
    for palabra in servicios_validos:
        if palabra in servicio:
            return reemplazos_servicio.get(palabra, palabra)
    return None

# --- PASO 4: Aplicar limpieza servicio ---
df['servicio_limpio'] = df['SERVICIO'].apply(limpiar_servicio)

print(df['servicio_limpio'])

0      Telefono
1      Telefono
2      Telefono
3      Telefono
4      Telefono
         ...   
456    Telefono
457    Telefono
458    Telefono
459    Telefono
460    Telefono
Name: servicio_limpio, Length: 461, dtype: object


**Solo si se requiere descargar ejecutar la siguiente casilla**

In [38]:
# Guardar el DataFrame modificado como archivo Excel
#df.to_excel('Servicio-Limpio.xlsx', index=False)

# Descargar el archivo a tu computadora
#from google.colab import files
#files.download('Servicio-Limpio.xlsx')


# **Crear hoja REGISTRO para EVALUADAS**

Ejecutar solo la primera celda, la última es para descargar el excel

In [39]:
# Renombramos columnas para facilitar el trabajo
df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)

# Aplicamos el mapeo
df['tipo_contacto'] = df['servicio_limpio']

# Capitalizamos la primera letra de cada tipo_contacto
df['tipo_contacto'] = df['tipo_contacto'].str.capitalize()
df['tipo_contacto'] = df['tipo_contacto'].replace('Whatsapp', 'WhatsApp')

# Columna fija
df['tipo_de_acción'] = 'Evaluada'

# Agrupamos por fecha, empresa y tipo_contacto (canal)
df = df.groupby(['empresa', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

#Pasar a minuscula
df.rename(columns={'EMPRESA': 'empresa'}, inplace=True)
df['empresa'] = df['empresa'].str.lower()

# Añadimos columna de tipo_de_acción (ya está aplicada arriba, pero por si acaso)
df['tipo_de_acción'] = 'Evaluada'

# Reordenamos columnas
df = df[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]


# Mostramos resultado
df

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Telefono,Evaluada,29
1,acer,Web,Evaluada,7
2,adeslas,Telefono,Evaluada,7
3,adeslas,Web,Evaluada,2
4,aeda homes,Telefono,Evaluada,11
...,...,...,...,...
63,ups,Telefono,Evaluada,1
64,viatris,Telefono,Evaluada,5
65,whirlpool,Telefono,Evaluada,2
66,xiaomi,Telefono,Evaluada,1


Extraer hoja registro

In [40]:
# Guardar el DataFrame modificado como archivo Excel
df.to_excel('Registro.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Registro.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# -----------------------------------------------------------------------------------------------

# ----------------------------------------------------------------------------------------------------

# Cargar un archivo del histórico distinto al último creado.

**Cuidado**, el bloque "Generar acumulativo..." debe ejecutarse manualmente punto a punto para que no tome el último  archivo creado del histórico sino este que estamos escogiendo manualmente.

In [ ]:
import os
import glob
import pandas as pd

# Ruta base donde están los archivos históricos
ruta_base = "/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Histórico"

# Buscar todos los archivos del historial
archivos = sorted(glob.glob(os.path.join(ruta_base, "Acumulativo_Recuento_historico_*.xlsx")))

# Mostrar lista con índice
print("📁 Archivos históricos disponibles:")
for i, archivo in enumerate(archivos):
    print(f"{i+1}. {os.path.basename(archivo)}")

# Elegir índice del archivo a cargar
indice = int(input("\n👉 Ingresá el número del archivo que querés cargar: ")) - 1

# Verificación
if 0 <= indice < len(archivos):
    archivo_seleccionado = archivos[indice]
    df_final_actualizado = pd.read_excel(archivo_seleccionado)
    print(f"\n✅ Archivo cargado: {archivo_seleccionado}")
else:
    print("❌ Índice fuera de rango.")


📁 Archivos históricos disponibles:
1. Acumulativo_Recuento_historico_2025-05-13_1.xlsx
2. Acumulativo_Recuento_historico_2025-05-13_2.xlsx
3. Acumulativo_Recuento_historico_2025-05-13_3.xlsx
4. Acumulativo_Recuento_historico_2025-05-13_4.xlsx
5. Acumulativo_Recuento_historico_2025-05-14_1.xlsx
6. Acumulativo_Recuento_historico_2025-05-14_2.xlsx


KeyboardInterrupt: Interrupted by user

# -----------------------------------------------------------------------------------------------------

# **Generar acumulativo para ver progreso del RECUENTO**

**Generar histórico acumaltivo**

**Solo ejecutar una vez!!**
Toma los valores de realizada y evaluada para ir actualizando los resultados.

**Cargar acumulativo histórico**

In [102]:
import os
import glob
import pandas as pd
from datetime import datetime

# Buscar el último archivo guardado
archivos = sorted(glob.glob(os.path.join(ruta_base, "Acumulativo_Recuento_historico_*.xlsx")))

if archivos:
    archivo_historico = archivos[-1]
    df_historico = pd.read_excel(archivo_historico)
    print(f"✅ Último histórico cargado desde: {archivo_historico}")
else:
    # Crear histórico vacío con las columnas necesarias
    columnas_clave = ['empresa', 'Inicio', 'Fin']
    columnas_actualizar = [col for col in df_final.columns if col.endswith('_Realizada') or col.endswith('_Evaluada')]
    df_historico = pd.DataFrame(columns=columnas_clave + columnas_actualizar)
    print("⚠️ No hay acumulativos anteriores. Se parte de vacío.")

⚠️ No hay acumulativos anteriores. Se parte de vacío.


In [103]:
# Limpiar espacios en nombres de columnas (importante antes del melt)
df2.columns = df2.columns.str.strip()

# También asegurarse que los valores en la columna 'Empresa' no tengan espacios
df2['Empresa'] = df2['Empresa'].str.strip()

# Verificamos que los nombres estén limpios y coincidan con 'canales'
canales = ['Telefono', 'Mail/formulario', 'Web', 'Facebook', 'Twitter', 'Chat', 'WhatsApp']


In [104]:
# Filtrar columnas a actualizar
columnas_actualizar = [col for col in df_historico.columns if col.endswith('_Realizada') or col.endswith('_Evaluada')]

# Asegurarse de que los índices clave sean iguales antes de sumar
df_historico['empresa'] = df_historico['empresa'].str.strip().str.lower()
df_final['empresa'] = df_final['empresa'].str.strip().str.lower()

# Definir columnas clave
columnas_clave = ['empresa', 'Inicio', 'Fin']

# Hacemos merge para acumular
df_actualizado = pd.merge(
    df_historico,
    df_final[columnas_clave + columnas_actualizar],
    on=columnas_clave,
    how='outer',
    suffixes=('', '_nuevo')
)

# Sumar columnas *_Realizada y *_Evaluada
for col in columnas_actualizar:
    col_nuevo = f"{col}_nuevo"
    if col_nuevo in df_actualizado.columns:
        df_actualizado[col] = df_actualizado[col].fillna(0) + df_actualizado[col_nuevo].fillna(0)
        df_actualizado.drop(columns=col_nuevo, inplace=True)

# Seleccionar solo columnas *_realizada, *_evaluada, *_total y claves
columnas_filtradas = columnas_clave + [col for col in df_actualizado.columns if col.lower().endswith(('_realizada', '_evaluada', '_total'))]

df_actualizado = df_actualizado[columnas_filtradas]

df_actualizado

<ipython-input-104-754d7becf17a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['empresa'] = df_final['empresa'].str.strip().str.lower()
<ipython-input-104-754d7becf17a>:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_actualizado[col] = df_actualizado[col].fillna(0) + df_actualizado[col_nuevo].fillna(0)
<ipython-input-104-754d7becf17a>:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,Facebook_Evaluada,Twitter_Realizada,Twitter_Evaluada,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,volotea,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,vueling,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,whirlpool,2025-05-05,2025-06-27,7.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,xiaomi,2025-05-05,2025-06-27,6.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Obtener total general, evaluadas y realizadas**

In [105]:
# Identificamos columnas por tipo
cols_realizada = [col for col in df_actualizado.columns if col.endswith('_Realizada')]
cols_evaluada = [col for col in df_actualizado.columns if col.endswith('_Evaluada')]
cols_total = [col for col in df_actualizado.columns if col.endswith('_Total')]

# Creamos las columnas de suma total
df_actualizado['Total_realizadas'] = df_actualizado[cols_realizada].sum(axis=1)
df_actualizado['Total_evaluadas'] = df_actualizado[cols_evaluada].sum(axis=1)
df_actualizado['Total_general'] = df_actualizado[cols_total].sum(axis=1)

df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,Facebook_Evaluada,Twitter_Realizada,Twitter_Evaluada,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,40.0,8.0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Calcular días restantes**

In [107]:
import numpy as np
from datetime import datetime
from workalendar.europe.spain import Catalonia  # o Spain, SpainCatalonia, etc.

# Asegurar que la columna Fin está en formato datetime
df_actualizado['Fin'] = pd.to_datetime(df_actualizado['Fin'])

# Fecha de hoy
hoy = np.datetime64(datetime.today().date(), 'D')

# Convertir 'Fin' a datetime64[D] para que funcione con busday_count
fechas_fin = df_actualizado['Fin'].values.astype('datetime64[D]')

# Obtener feriados nacionales + Madrid del año actual y siguiente
cal = Catalonia()
año_actual = datetime.today().year
feriados = cal.holidays(año_actual) + cal.holidays(año_actual + 1)
feriados_np = np.array([np.datetime64(d[0], 'D') for d in feriados])

# Calcular días hábiles restantes excluyendo feriados
df_actualizado['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)

# Visualizar
df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,...,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar,Dias_laborables_restantes
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,40.0,8.0,-40.0,-8.0,29
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,2.0,-12.0,-2.0,29
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,-3.0,29
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29


**Obtener columnas por evaluar y realizar**

In [106]:
df_actualizado['Por_realizar'] = df_actualizado['Total_general'] - df_actualizado['Total_realizadas']
df_actualizado['Por_evaluar'] = df_actualizado['Total_general'] - df_actualizado['Total_evaluadas']

df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,...,Twitter_Evaluada,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,40.0,8.0,-40.0,-8.0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,-12.0,-2.0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,-3.0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**A realizar y a evaluar según los días restantes**

In [108]:
# Asegurarse de evitar división por cero o días negativos
df_actualizado['Dias_laborables_restantes'] = df_actualizado['Dias_laborables_restantes'].clip(lower=1)

# Calcular valores por día
df_actualizado['A_realizar_por_dia'] = ((df_actualizado['Total_general'] - df_actualizado['Total_realizadas']) / df_actualizado['Dias_laborables_restantes']).round(0).astype(int)
df_actualizado['A_evaluar_por_dia'] = ((df_actualizado['Total_general'] - df_actualizado['Total_evaluadas']) / df_actualizado['Dias_laborables_restantes']).round(0).astype(int)

df_actualizado

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,...,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar,Dias_laborables_restantes,A_realizar_por_dia,A_evaluar_por_dia
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,40.0,8.0,-40.0,-8.0,29,-1,0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,12.0,2.0,-12.0,-2.0,29,0,0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,-3.0,29,0,0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,volotea,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
90,vueling,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
91,whirlpool,2025-05-05,2025-06-27,7.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,0.0,-9.0,0.0,29,0,0
92,xiaomi,2025-05-05,2025-06-27,6.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,1.0,-8.0,-1.0,29,0,0


**Almacenar en Drive el histórico que alimentará al siguiente**

In [109]:
fecha = datetime.now().strftime("%Y-%m-%d")
contador = len(glob.glob(os.path.join(ruta_base, f"Acumulativo_Recuento_historico_{fecha}_*.xlsx")))
nombre_archivo = f"Acumulativo_Recuento_historico_{fecha}_{contador + 1}.xlsx"
ruta_guardado = os.path.join(ruta_base, nombre_archivo)

df_actualizado.to_excel(ruta_guardado, index=False)

print(f"✅ Histórico actualizado y guardado en: {ruta_guardado}")

✅ Histórico actualizado y guardado en: /content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Histórico/Acumulativo_Recuento_historico_2025-05-16_1.xlsx


**Descargar excel**

In [110]:
# Guardar el DataFrame modificado como archivo Excel
df_actualizado.to_excel('Acumulativo Recuento por TIPO DE CONTACTO.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Acumulativo Recuento por TIPO DE CONTACTO.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ------------------------------------------------------------------------------------------------------------------